Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [2]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

67921/67921 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [4]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts trainn samples:', train_data[:10])

number of samples: 3650
number of train samples: 3000
number of test samples: 650
firsts trainn samples: [20.7 17.9 18.8 14.6 15.8 15.8 15.8 17.4 21.8 20. ]


<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [22]:
# windowing function
def create_windowed_dataset(data, past, future):
    X, y = [], []
    for i in range(len(data) - past - future + 1):
        X.append(data[i:i+past])
        y.append(data[i+past+future-1])
    return np.array(X), np.array(y)

In [23]:
past, future = (5, 2)
X_train, y_train = create_windowed_dataset(train_data, past, future)
X_test, y_test = create_windowed_dataset(test_data, past, future)

# Comprobamos el resultado
for i in range(4):
    print(f"x[{i}] : {X_train[i]} , y[{i}]: {y_train[i]}")

x[0] : [20.7 17.9 18.8 14.6 15.8] , y[0]: 15.8
x[1] : [17.9 18.8 14.6 15.8 15.8] , y[1]: 17.4
x[2] : [18.8 14.6 15.8 15.8 15.8] , y[2]: 21.8
x[3] : [14.6 15.8 15.8 15.8 17.4] , y[3]: 20.0


<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [24]:
# Definir la forma de los datos de entrada
input_shape = (past, 1)

# Crear el modelo
inputs = keras.layers.Input(shape=input_shape)
x = layers.GRU(32, return_sequences=True)(inputs)
x = layers.GRU(16)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 5, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 5, 32)          │         3,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 16)             │         2,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,777 (22.57 KB)

 Trainable params: 5,777 (22.57 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 121.7312 - val_loss: 95.0040
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 67.7681 - val_loss: 72.0983
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 53.7380 - val_loss: 61.3019
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 44.0780 - val_loss: 53.1569
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 39.5156 - val_loss: 46.5279
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 34.3784 - val_loss: 41.0231
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 29.5656 - val_loss: 36.4217
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 27.1716 - val_loss: 32.6993
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 25.4592 - val_loss: 29.5532
Epoch 10/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 22.7992 - val_loss: 27.0360
Epoch 11/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 21.1036 - val_loss: 24.9706
Epoch 12/200
38/38 ━━━━━━━━━━━━━━━━━━━━

In [26]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6.7693 
Test Loss: 7.018261432647705


<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [31]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df_multi = df[['Temp', 'portion_year']].copy()

## train - test split
train_data = df_multi.iloc[:3000].copy()
test_data = df_multi.loc[3000:, :].copy()

In [32]:
## Create windows
def create_multi_windowed_dataset(data, past, future):
    X, y = [], []
    for i in range(len(data) - past - future + 1):
        X.append(data.iloc[i:i+past].values)
        y.append(data.iloc[i+past+future-1]['Temp'])
    return np.array(X), np.array(y)

X_train, y_train = create_multi_windowed_dataset(train_data, past, future)
X_test, y_test = create_multi_windowed_dataset(test_data, past, future)

In [39]:
input_shape = (past, X_train.shape[2])

inputs = keras.layers.Input(shape=input_shape)
x = layers.GRU(64, return_sequences=True)(inputs)
x = layers.GRU(32, return_sequences=True)(x)
x = layers.GRU(32)(x)
outputs = layers.Dense(1)(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 5, 2)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_9 (GRU)                     │ (None, 5, 64)          │        13,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_10 (GRU)                    │ (None, 5, 32)          │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_11 (GRU)                    │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,833 (112.63 KB)

 Trainable params: 28,833 (112.63 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 88.4523 - val_loss: 34.8146
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 24.1789 - val_loss: 23.7939
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 19.8743 - val_loss: 19.9926
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 17.0507 - val_loss: 18.1949
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 16.4177 - val_loss: 17.3386
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 16.3552 - val_loss: 17.0362
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 16.7369 - val_loss: 16.9938
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 17.5883 - val_loss: 16.6745
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 14.5589 - val_loss: 13.4079
Epoch 10/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 12.0736 - val_loss: 11.6620
Epoch 11/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 10.5888 - val_loss: 10.7383
Epoch 12/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 

In [41]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.8779 
Test Loss: 6.273911952972412


<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


a, b, d

* a recibe una secuencia de palabras y se mapea a una única etiqueta.
* b recibe una secuencia de características de la voz y da uan decisión de verificado o no.
* d recibe una secuencia de características o notas de una piezas y da una única etiqueta de autor.

Para c sería más adecuado many-to-many.

<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.


a, b, c, d